In [7]:
# python env: atom2env_dev_v2

import json                                                                                                                             
import monty.json
import sys
 
from pathlib import Path
import pickle

import shutil
import os
import numpy as np
import pandas as pd
import pickle
import traceback as tb
import sys
import warnings
warnings.filterwarnings("ignore")

from pymatgen.core.structure import Structure
from abipy.core.structure import Structure as AbiStructure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from atomate2.abinit.files import del_gzip_files
from atomate2.abinit.jobs.response import DdkMaker, DdeMaker, DteMaker
from atomate2.abinit.jobs.core import StaticMaker
from atomate2.abinit.flows.dfpt import ShgFlowMaker
from atomate2.abinit.powerups import (
     update_user_abinit_settings,
     update_factory_kwargs,
     update_generator_attributes,
     update_user_kpoints_settings,
     update_taskdoc,
     append_clean_flow
     )
from atomate2.abinit.sets.base import as_pseudo_table
from fireworks import LaunchPad
from importlib import reload
from jobflow import run_locally, Flow
from jobflow.managers.fireworks import flow_to_workflow
from tqdm import tqdm

In [8]:
# Import sys and set the path of the customized modules
import sys
sys.path.append('/home/vtrinquet/Documents/Doctorat/JNB_Scripts_Clusters/Custom_Modules/')

# Import importlib.reload to be able to reload dynamically modified modules
from importlib import reload

import modif_spec_atomate2 as mod_spec
mod_spec = reload(mod_spec)

In [9]:
def test_chksymbreak(structure, kgrid="Symmetric"):
    # TEST CHKSYMBREAK
    from maggma.stores import MemoryStore
    from jobflow import JobStore
    import multiprocessing
    test_chksymbreak_passed = multiprocessing.Value('i', 1)
    def test_chksymbreak():
        docs_store = MemoryStore()
        store = JobStore(docs_store)
        test_chksymbreak_job = StaticMaker().make(structure=structure)
        test_chksymbreak_job = update_user_abinit_settings(test_chksymbreak_job, {'chksymbreak': 1})
        test_chksymbreak_job = update_factory_kwargs(test_chksymbreak_job, {'shift_mode': kgrid})
        try:
            run_locally(test_chksymbreak_job, store=store, create_folders=True, root_dir='test_chksymbreak', raise_immediately=True)
        except (Exception, ValueError):
            print("chksymbreak triggered")
            test_chksymbreak_passed.value = 0
    p = multiprocessing.Process(target=test_chksymbreak, name="Foo")
    p.start()
    # Wait a maximum of 5 seconds for foo
    # Usage: join([timeout in seconds])
    p.join(5)

    # If thread is active
    if p.is_alive():
        print("job is running... let's kill it...")
        # Terminate foo
        p.terminate()
        p.join()
    
    return test_chksymbreak_passed

In [10]:
cur_v = int(Path(os.getcwd()).parent.name.split("_")[0])
print(cur_v)

19


# LaunchPad

In [11]:
path_df_selected = f'../data/df_selected_v{cur_v}.pkl.gz'
df_selected = pd.read_pickle(path_df_selected)
print(f"{df_selected.shape = }")

df_selected.shape = (386, 11)


In [12]:
#   Paths to the candidates datasets
path_df_selected = f'../data/df_selected_v{cur_v}.pkl.gz'
df_selected = pd.read_pickle(path_df_selected)

iter = 0
count = 0
count_low=1     # included ; done: 1,
count_high=550    # included ; done:  550,
count_chksymbreak_passed = 0 # to count the nb of materials which do not trigger chksymbreak
list_idxs_triggered = []

#   Iterating over the candidates to add them in the Launchpad 
for idx, s in tqdm(zip(df_selected.index.values, df_selected['structure']), total=len(df_selected)):

    count += 1
    if count<=count_low-1:
        continue
    if count>count_high:
        break
    print(count)

    # Initializes the standardized primitive MP structure
    structure = AbiStructure.from_dict(s)
    structure = structure.abi_sanitize(primitive_standard=True)
    # not applied in v18, but will be applied from now on to avoid running structures that fail at the DTE level 
    # which cannot handle an uneven number of electrons because nband must be the number of valence bands precisely
    pseudos = as_pseudo_table(pseudos="ONCVPSP-LDA-SR-PDv0.4:standard")
    if structure.num_valence_electrons(pseudos=pseudos)%2:
        print("Uneven number of valence electrons. Let's skip this structure")
        continue
    structure.__class__ = Structure

    # TEST CHKSYMBREAK
    test_chksymbreak_passed = test_chksymbreak(structure, kgrid="Symmetric")
    if not test_chksymbreak_passed.value:
        list_idxs_triggered.append(idx)
        continue

    # kill any existing abinit process bc not killed by p.terminate() and then make everything crash
    import os
    os.system("ps -C abinit -o pid=|xargs kill -9")

    count_chksymbreak_passed += 1
    import gc
    gc.collect()


    # Create a Dfpt WF to compute the SHG tensor
    shg_flow = ShgFlowMaker().make(structure=structure)

    shg_flow = update_generator_attributes(shg_flow, {'pseudos': 'ONCVPSP-LDA-SR-PDv0.4:standard'}, name_filter='Scf calculation')

    shg_flow = update_taskdoc(shg_flow, {"files_to_store": []}, name_filter="Scf calculation")
    shg_flow = update_taskdoc(shg_flow, {"files_to_store": ["DDB", "POT"]}, class_filter=DdkMaker)
    shg_flow = update_taskdoc(shg_flow, {"files_to_store": ["DDB", "POT"]}, class_filter=DdeMaker)
    shg_flow = update_taskdoc(shg_flow, {"files_to_store": ["DDB", "POT"]}, class_filter=DteMaker)
    shg_flow = update_taskdoc(shg_flow, {"files_to_store": ["DDB"]}, name_filter="Merge DDB")


    torun_flow = append_clean_flow(
        shg_flow,
        delete=True,
        exclude_files_from_zip=None,
        exclude_files_from_del=['*POT*']
    )

    # Let's add a metadata to recover it easily from the MongoDb afterwards with {"spec._tasks.job.metadata.Label": "HSE_Etot(x)"}
    if df_selected.loc[idx]['nsites'] > 25:
        is_light = "heavy"
    else:
        is_light = "light"
    torun_flow.update_metadata({"Batch": f"shg_hg_v{cur_v}", "material_id": f"{idx}", "is_light": is_light})

    # convert the flow to a fireworks WorkFlow object
    wf = flow_to_workflow(torun_flow)

    # # submit the workflow to the FireWorks launchpad
    # lpad = LaunchPad.auto_load()
    # lpad.add_wf(wf)

    gc.collect()
shutil.rmtree('test_chksymbreak')


  0%|          | 0/386 [00:00<?, ?it/s]

1
2025-01-05 15:51:22,930 INFO Started executing jobs locally
2025-01-05 15:51:23,140 INFO Starting job - Scf calculation (5c1519b2-8a90-4fee-8514-f1a90d82fd93)
job is running... let's kill it...
2025-01-05 15:51:28,541 INFO Added a workflow. id_map: {-11: 94495, -10: 94496, -9: 94497, -8: 94498, -7: 94499, -6: 94500, -5: 94501, -4: 94502, -3: 94503, -2: 94504, -1: 94505}


  0%|          | 1/386 [00:05<37:12,  5.80s/it]

2
2025-01-05 15:51:28,721 INFO Started executing jobs locally
2025-01-05 15:51:28,729 INFO Starting job - Scf calculation (254ec17b-4ace-41c9-a595-e9105036486a)
job is running... let's kill it...
2025-01-05 15:51:34,259 INFO Added a workflow. id_map: {-22: 94506, -21: 94507, -20: 94508, -19: 94509, -18: 94510, -17: 94511, -16: 94512, -15: 94513, -14: 94514, -13: 94515, -12: 94516}


  1%|          | 2/386 [00:11<36:45,  5.74s/it]

...

 99%|█████████▉| 384/386 [33:21<00:11,  5.63s/it]

385
2025-01-05 16:24:44,916 INFO Started executing jobs locally
2025-01-05 16:24:44,924 INFO Starting job - Scf calculation (f388871d-71fa-4ee2-89fa-0fcd4288196e)
job is running... let's kill it...
2025-01-05 16:24:50,415 INFO Added a workflow. id_map: {-3872: 98357, -3871: 98358, -3870: 98359, -3869: 98360, -3868: 98361, -3867: 98362, -3866: 98363, -3865: 98364, -3864: 98365, -3863: 98366, -3862: 98367}


100%|█████████▉| 385/386 [33:27<00:05,  5.64s/it]

386
2025-01-05 16:24:50,575 INFO Started executing jobs locally
2025-01-05 16:24:50,582 INFO Starting job - Scf calculation (8a603aac-96a5-43e3-a80a-75cb87aeb4e1)
job is running... let's kill it...
2025-01-05 16:24:56,073 INFO Added a workflow. id_map: {-3883: 98368, -3882: 98369, -3881: 98370, -3880: 98371, -3879: 98372, -3878: 98373, -3877: 98374, -3876: 98375, -3875: 98376, -3874: 98377, -3873: 98378}


100%|██████████| 386/386 [33:33<00:00,  5.22s/it]


In [13]:
print(f"Out of {len(df_selected)} selected materials, {count_chksymbreak_passed} passed the chksymbreak test")

Out of 386 selected materials, 353 passed the chksymbreak test


In [ ]:
kgrid = "Gamma"

#   Paths to the candidates datasets
df_selected_bis = df_selected.filter(list_idxs_triggered, axis=0)

iter = 0
count = 0
count_low=1     # included ; done: 1,
count_high=350    # included ; done:  250,
count_chksymbreak_passed = 0 # to count the nb of materials which do not trigger chksymbreak
list_idxs_triggered = []

#   Iterating over the candidates to add them in the Launchpad 
for idx, s in tqdm(zip(df_selected_bis.index.values, df_selected_bis['structure']), total=len(df_selected_bis)):

    count += 1
    if count<=count_low-1:
        continue
    if count>count_high:
        break
    print(count)

    # Initializes the standardized primitive MP structure
    structure = AbiStructure.from_dict(s)
    structure = structure.abi_sanitize(primitive_standard=True)
    structure.__class__ = Structure

    # TEST CHKSYMBREAK
    test_chksymbreak_passed = test_chksymbreak(structure, kgrid=kgrid)
    if not test_chksymbreak_passed.value:
        list_idxs_triggered.append(idx)
        continue

    # kill any existing abinit process bc not killed by p.terminate() and then make everything crash
    import os
    os.system("ps -C abinit -o pid=|xargs kill -9")

    count_chksymbreak_passed += 1
    import gc
    gc.collect()


    # Create a Dfpt WF to compute the SHG tensor
    shg_flow = ShgFlowMaker().make(structure=structure)

    shg_flow = update_generator_attributes(shg_flow, {'pseudos': 'ONCVPSP-LDA-SR-PDv0.4:standard'}, name_filter='Scf calculation')
    shg_flow = update_factory_kwargs(shg_flow, {'shift_mode': kgrid}, name_filter='Scf calculation')

    shg_flow = update_taskdoc(shg_flow, {"files_to_store": []}, name_filter="Scf calculation")
    shg_flow = update_taskdoc(shg_flow, {"files_to_store": ["DDB", "POT"]}, class_filter=DdkMaker)
    shg_flow = update_taskdoc(shg_flow, {"files_to_store": ["DDB", "POT"]}, class_filter=DdeMaker)
    shg_flow = update_taskdoc(shg_flow, {"files_to_store": ["DDB", "POT"]}, class_filter=DteMaker)
    shg_flow = update_taskdoc(shg_flow, {"files_to_store": ["DDB"]}, name_filter="Merge DDB")


    torun_flow = append_clean_flow(
        shg_flow,
        delete=True,
        exclude_files_from_zip=None,
        exclude_files_from_del=['*POT*']
    )

    # Let's add a metadata to recover it easily from the MongoDb afterwards with {"spec._tasks.job.metadata.Label": "HSE_Etot(x)"}
    if df_selected.loc[idx]['nsites'] > 25:
        is_light = "heavy"
    else:
        is_light = "light"
    torun_flow.update_metadata({"Batch": f"shg_hg_v{cur_v}", "material_id": f"{idx}", "is_light": is_light})

    # convert the flow to a fireworks WorkFlow object
    wf = flow_to_workflow(torun_flow)

    # # submit the workflow to the FireWorks launchpad
    # lpad = LaunchPad.auto_load()
    # lpad.add_wf(wf)

    gc.collect()
shutil.rmtree('test_chksymbreak')


  0%|          | 0/17 [00:00<?, ?it/s]

1
2025-01-05 16:24:56,369 INFO Started executing jobs locally
2025-01-05 16:24:56,387 INFO Starting job - Scf calculation (270ced07-d1b1-4764-8540-4adc9b75d3b5)
chksymbreak triggered


  6%|▌         | 1/17 [00:00<00:05,  3.04it/s]

2
2025-01-05 16:24:56,690 INFO Started executing jobs locally
2025-01-05 16:24:56,700 INFO Starting job - Scf calculation (af4d17fb-f0a6-4eca-8c21-7b43ea11b209)
chksymbreak triggered


 12%|█▏        | 2/17 [00:00<00:05,  2.60it/s]

...

 88%|████████▊ | 15/17 [00:22<00:01,  1.01it/s]

16
2025-01-05 16:25:18,378 INFO Started executing jobs locally
2025-01-05 16:25:18,388 INFO Starting job - Scf calculation (91175538-c28b-422b-bf79-9e14f397fe4a)
job is running... let's kill it...
2025-01-05 16:25:23,924 INFO Added a workflow. id_map: {-3927: 98412, -3926: 98413, -3925: 98414, -3924: 98415, -3923: 98416, -3922: 98417, -3921: 98418, -3920: 98419, -3919: 98420, -3918: 98421, -3917: 98422}


 94%|█████████▍| 16/17 [00:27<00:02,  2.42s/it]

17
2025-01-05 16:25:24,091 INFO Started executing jobs locally
2025-01-05 16:25:24,101 INFO Starting job - Scf calculation (3c2423da-5979-43ac-9e2f-8a8421a685e6)
job is running... let's kill it...
2025-01-05 16:25:29,689 INFO Added a workflow. id_map: {-3938: 98423, -3937: 98424, -3936: 98425, -3935: 98426, -3934: 98427, -3933: 98428, -3932: 98429, -3931: 98430, -3930: 98431, -3929: 98432, -3928: 98433}


100%|██████████| 17/17 [00:33<00:00,  1.97s/it]


In [15]:
print(f"Out of {len(df_selected_bis)} selected materials, {count_chksymbreak_passed} passed the chksymbreak test")

Out of 17 selected materials, 5 passed the chksymbreak test


In [16]:
df_selected_bis.shape

(17, 11)